In [1]:
from itertools import cycle
import numpy as np
from ipywidgets import IntSlider, FloatSlider, SelectionSlider

import plotlymath as plt
from myutils import interact, latex


In [2]:
plt.set_defaults(margin=(25,0,40,50))
colors = plt.plotly.colors.DEFAULT_PLOTLY_COLORS

In [3]:
def discrete_linear_interactive():
    xy_max = 10
    line_length = 1000 * xy_max
    max_steps = 50
    labelpos = np.array([[0, -1], [1, 0]]) * 20
    A = np.array([[1.1, 0], [0, 0.7]]) # Default initial matrix
    trace_color = cycle(colors)
    figure, plot = plt.make_figure(widget=True)
    figure.layout.hovermode = "closest"
    figure.layout.showlegend = False
    figure.layout.update(width=750, height=600)
    figure.layout.xaxis.domain = (0, 0.75)
    plot.axes_labels("$x$", "$y$")
    plot.axes_ranges((-xy_max, xy_max), (-xy_max, xy_max), scale=(1, 1))
    x = np.linspace(-xy_max, xy_max, 4 * xy_max + 1)
    y = np.linspace(-xy_max, xy_max, 4 * xy_max + 1)
    x, y = np.meshgrid(x, y)
    points = np.array((x.flatten(), y.flatten())).transpose()
    plot.grid = plt.points(points, opacity=0, hoverinfo="none")

    evalues = np.round(np.linspace(-1.5, 1.5, 61), 2)
    @interact(v1=IntSlider(0, 0, 360, 5, description="Eigenvector 1"), 
              l1=SelectionSlider(options=evalues, value=1.1, description=r"$\lambda_1$"), 
              v2=IntSlider(90, 0, 360, 5, description="Eigenvector 2"), 
              l2=SelectionSlider(options=evalues, value=0.7, description=r"$\lambda_2$"))
    def update(v1, l1, v2, l2):
        nonlocal trace_color
        trace_color = cycle(colors)
        if (v1 - v2) % 180 == 0:
            A[:,:] = np.zeros((2, 2))
            with figure.batch_update():
                error = "Invalid!<br>Eigenvectors must<br>not overlap"
                plot.matrix = plt.text(error, (0.8, 0.8), size=12, paper=True)
                figure.data[1].update(x=[], y=[])
                figure.data[2].update(x=[], y=[])
                figure.data[3].update(x=[], y=[])
                figure.layout.annotations[1].text = ""
                figure.layout.annotations[2].text = ""
            return
        T = np.cos(np.array([[v1, v2], [90 - v1, 90 - v2]]) * np.pi/180)
        A[:,:] = T @ np.diag([l1, l2]) @ np.linalg.inv(T)
        with figure.batch_update():
            matrix = fr"$\quad\text{{Matrix:}}\\[4pt]{latex(A, round=3)}$"
            plot.matrix = plt.text(matrix, (0.8, 0.8), size=24, paper=True)
            if abs(l1) == abs(l2):
                NaN = np.array((np.nan, np.nan), dtype=float)
                eigen = T * line_length
                eigen = (-eigen[:,0], eigen[:,0], NaN, -eigen[:,1], eigen[:,1])
                plot.dominant = plt.lines(eigen, color="yellow", line_width=5)
            else:
                eigen = (T[:,0] if abs(l1) > abs(l2) else T[:,1]) * line_length
                plot.dominant = plt.lines((-eigen, eigen), color="yellow", line_width=5)
            eigen = T[:,0] * line_length
            color = "darkgreen" if l1 < 1 else "darkred" if l1 > 1 else "gray"
            plot.evec1 = plt.lines((-eigen, eigen), color=color, line_width=3)
            eigen = T[:,0] * (0.7 * xy_max)
            xshift, yshift = labelpos @ T[:,0] * (0.35 if 90 <= v1 < 270 else 1)
            label = fr"$\lambda_1 = {latex(l1)}$"
            plot.eval1 = plt.text(label, eigen, color=color, size=24, 
                    textangle=90 - (v1 + 90) % 180, xshift=xshift, yshift=yshift)
            eigen = T[:,1] * line_length
            color = "darkgreen" if l2 < 1 else "darkred" if l2 > 1 else "gray"
            plot.evec2 = plt.lines((-eigen, eigen), color=color, line_width=3)
            eigen = T[:,1] * (0.7 * xy_max)
            xshift, yshift = labelpos @ T[:,1] * (0.35 if 90 <= v2 < 270 else 1)
            label = fr"$\lambda_2 = {latex(l2)}$"
            plot.eval2 = plt.text(label, eigen, color=color, size=24, 
                    textangle=90 - (v2 + 90) % 180, xshift=xshift, yshift=yshift)
            figure.data = figure.data[:4]
    update(0, 1.1, 90, 0.7)

    options = dict(mode="markers+lines", line_color="lightgray", hoverinfo="skip")
    def click_handler(trace, points, state):
        solution = [(points.xs[0], points.ys[0])]
        for t in range(max_steps):
            solution.append(A @ solution[-1])
        figure.add_trace(plt.points(solution, color=next(trace_color), **options))
    figure.data[0].on_click(click_handler)
    return figure


# In the interactive below, 
the sliders for “Eigenvector 1” and “Eigenvector 2” control the angles of the two eigen-lines. The other two sliders control the corresponding eigenvalues. Manipulating all four of these allows you to get any diagonalizable 2×2 matrix with real eigenvalues. 

(For matrices with non-real eigenvalues, see the second interactive, further down.) 

Clicking any point in the state space will show the behavior of a simulation starting at that initial state. 


In [4]:
discrete_linear_interactive()

interactive(children=(IntSlider(value=0, description='Eigenvector 1', max=360, step=5), SelectionSlider(descri…

FigureWidget({
    'data': [{'hoverinfo': 'none',
              'marker': {'size': 8},
              'mode': '…

In [6]:
def rotation(d):
    return np.cos((np.array([[0, 270], [90, 0]]) - d) * (np.pi/180))

def complex_discrete_linear_interactive():
    xy_max = 10
    line_length = 0.3 * xy_max
    max_steps = 50
    A = np.array([[0, 1], [-1, 0]], dtype=float) # Default initial matrix
    trace_color = cycle(colors)
    figure, plot = plt.make_figure(widget=True)
    figure.layout.hovermode = "closest"
    figure.layout.showlegend = False
    figure.layout.update(width=750, height=600)
    figure.layout.xaxis.domain = (0, 0.75)
    plot.axes_labels("$x$", "$y$")
    plot.axes_ranges((-xy_max, xy_max), (-xy_max, xy_max), scale=(1, 1))
    x = np.linspace(-xy_max, xy_max, 4 * xy_max + 1)
    y = np.linspace(-xy_max, xy_max, 4 * xy_max + 1)
    x, y = np.meshgrid(x, y)
    points = np.array((x.flatten(), y.flatten())).transpose()
    plot.grid = plt.points(points, opacity=0, hoverinfo="none")

    angles = np.array((np.arange(-175, 0, 5), np.arange(5, 180, 5))).flatten()
    @interact(real=IntSlider(0, 0, 360, 5, description="Real part"), 
              rate=FloatSlider(value=1, min=0.5, max=1.5, step=0.02, 
                               description=r"$\lvert\lambda\rvert$"), 
              imaglen=FloatSlider(value=1, min=0.3, max=3, step=0.1, 
                                  description="Imag part"), 
              theta=SelectionSlider(options=angles, value=25, 
                                    description=r"$\arg(\lambda)$"))
    def update(real, rate, imaglen, theta):
        nonlocal trace_color
        trace_color = cycle(colors)
        T = rotation(real) * (1, imaglen)
        A[:,:] = T @ (rate * rotation(theta)) @ np.linalg.inv(T)
        with figure.batch_update():
            matrix = fr"$\quad\text{{Matrix:}}\\[4pt]{latex(A, round=3)}$"
            plot.matrix = plt.text(matrix, (0.8, 0.8), size=24, paper=True)
            eigen = latex(rate * np.exp(theta*np.pi/180*1j), round=3, conjpair=True)
            eigen = fr"$\text{{Eigenvalues:}}\\[4pt]\lambda = {eigen}$"
            plot.evalue = plt.text(eigen, (0.8, 0.6), size=24, paper=True)
            eigen = T[:,0] * line_length
            plot.evec1 = plt.lines(((0,0), eigen), color="black", line_width=3)
            eigen = T[:,1] * line_length
            plot.evec2 = plt.lines(((0,0), eigen), color="black", line_width=3)
            figure.data = figure.data[:3]
    update(0, 1, 1, 25)

    options = dict(mode="markers+lines", line_color="lightgray", hoverinfo="skip")
    def click_handler(trace, points, state):
        solution = [(points.xs[0], points.ys[0])]
        for t in range(max_steps):
            solution.append(A @ solution[-1])
        figure.add_trace(plt.points(solution, color=next(trace_color), **options))
    figure.data[0].on_click(click_handler)
    return figure


# In this second interactive, 
the matrix will always be a 2×2 matrix with a conjugate pair of non-real eigenvalues. The sliders for $\lvert\lambda\rvert$ and $\arg(\lambda)$ control the eigenvalue, in polar form: 
$$ \lambda = r e^{i \theta} \qquad \text{where } r = \lvert\lambda\rvert \text{ and } \theta = \arg(\lambda) $$

The sliders for “Real part” and “Imag part” control the eigenvector, in the following way: any eigenvector corresponding to the non-real eigenvalue $\lambda = a + bi$ will have the form $\vec{v} + i\vec{w}$ for some vectors $\vec{v}$ and $\vec{w}$ in $\mathbb{R}^2$. Furthermore, $\vec{v}$ and $\vec{w}$ can be chosen to be perpendicular, in which case the solutions to the discrete-time model $\vec{x}(n+1) = A \vec{x}$ will rotate (or spiral in/out) in an elliptical shape whose major and minor axes are the lines through $\vec{v}$ and $\vec{w}$. The “Real part” slider controls the direction of $\vec{v}$ (the real part of this eigenvector) as an angle. Then $\vec{w}$ will be chosen to be perpendicular to $\vec{v}$ (90° CCW from it). But $\vec{w}$ can have a length that is shorter or longer than $\vec{v}$. The “Imag part” slider controls this length, relative to the length of $\vec{v}$. So setting this to $1$ makes them the same length, meaning that the elliptical orbits of the simulations will actually be circles. Setting it to something greater than 1 makes $\vec{w}$ the major axis of the ellipses, and setting it to something less than 1 makes $\vec{w}$ the minor axis of the ellipses. 

Manipulating all four of these quantities allows you to get any 2×2 matrix with non-real eigenvalues. 

Once again, clicking any point in the state space will show the results of a simulation starting at that initial state. 


In [7]:
complex_discrete_linear_interactive()

interactive(children=(IntSlider(value=0, description='Real part', max=360, step=5), FloatSlider(value=1.0, des…

FigureWidget({
    'data': [{'hoverinfo': 'none',
              'marker': {'size': 8},
              'mode': '…